## Setup Most Popular Recommender

For the Most popular Recommender we will need two main things:
* An **Index Map** to map an item_id into an index (e.g. 1, 2, 7, 45, etc.)
* The **Item Scores** to define which items are, in fact, the most popular ones

In this notebook we will set up those two. However, the actual recommendation happens in `most_popular.py` that will answer to the BentoML api when requested.


### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from most_popular import MostPopularRecommender
from preprocessing import preprocess

### Acquire preprocessed Data

In [2]:
df = preprocess("Sample")

In [11]:
df.head(6)

,Timestamp,Clicked_Article,Click,User_Features,Article_List
0,1317513291,560620,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029..."
1,1317513291,565648,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029..."
2,1317513291,563115,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029..."
3,1317513292,552077,0,"[True, False, False, False, False, False, True...","[552077, 555224, 555528, 559744, 559855, 56029..."
4,1317513292,564335,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029..."
5,1317513292,565589,0,"[True, False, False, False, True, False, False...","[552077, 555224, 555528, 559744, 559855, 56029..."


### Index Map

First, we get all articles in a list

In [3]:
articles = df['Clicked_Article'].unique()

Then, we iterate over them creating a dictionary for the index map.

In [4]:
index_map = {}
idx = 1 # idx starts at 1 so that 0 is used for when the article is not found in the index map
for art in articles:
    index_map[str(art)] = idx
    idx+=1

### Item Score

For each article, we count how many times it has been clicked.

In [5]:
popular = df.loc[(df['Click']==1)].groupby('Clicked_Article').size().sort_values(ascending=False)
popular.head(5)

Clicked_Article
565747    27
565515    26
565822    25
555224    24
563115    23
dtype: int64

Now, using the Index Map, we associate each index with a value.

In [6]:
item_score = {'0': -1} 
#since 0 is used for when the article was not found in the index map, here it'll have the lowest value
for art in articles:
    item_score[str(index_map[str(art)])] = popular[art]

### Saving Dictionaries

In order to pass our **Index Map** and **Item Score** dictionaries to the model, we use BentoML. Thus, our recommender will load those dictionaries in order to make its recommendations.

The `pack()` function takes care of saving our dictionaries.

In [7]:
model = MostPopularRecommender()

In [8]:
model.pack("item_score", item_score)

In [9]:
model.pack("index_map", index_map)

After packing what our recommender will need, we can test it with a small sample

In [10]:
model.rank({'Timestamp': 123456789, 'Clicked_Article': 565822, 'Click': 1, 'User_Features': np.asarray([True,False,False,False,True]), 'Article_List': np.asarray([565648, 563115, 552077, 564335, 565589, 563938, 560290, 563643, 560620, 565822, 563787, 555528, 565364, 559855, 560518])})

[565822,
 565648,
 565589,
 565364,
 564335,
 563938,
 563787,
 563643,
 563115,
 560620,
 560518,
 560290,
 559855,
 555528,
 552077]